This document corresponds to the assignment for Week 3 of the IBM Data Science Professional Certificate Capstone Course.

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

We are going to use the requests and lxml libraries to parse the html table from wikipedia. We begin grabbing the page, then loading into an lxml document. The lxml library is helpful because we can use it to filter the table by the html tag. Using the library we are able to quickly parse the 3 column table by grabbing the column names before repeating with the actual data. We then convert this to a dictionary for pandas.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')
col = []
i = 0
for t in tr_elements[0]:
    i += 1
    name = t.text_content()
    name = name.replace('\n', '')
    col.append((name, []))

for j in range(1, len(tr_elements)):
    row = tr_elements[j]
    if len(row) != 3:
        break
    i = 0
    
    for t in row.iterchildren():
        data = t.text_content()
        col[i][1].append(data)
        i += 1

neighbor_dict = {title: column for title, column in col}

We need to load the dictionary into a dataframe. Then we are going to apply various transformations to make the data workable. First, we eliminate the newline characters. We then replace 'Not assigned' with NaN values to be used later. The DataFrame.combine_first function is used to default NaN Neighborhoods to the Borough name. Finally we drop any remaining NaNs. We also drop the extra column as the author is accustomed to the American spelling of 'Neighborhood'.

In [3]:
nbhd_df = pd.DataFrame(neighbor_dict)
nbhd_df.head()
#Clean up new lines
nbhd_df['Neighborhood'] = nbhd_df['Neighbourhood'].apply(lambda x: x.replace('\n', ''))
nbhd_df = nbhd_df.drop(['Neighbourhood'], axis=1)
nbhd_df = nbhd_df.replace('Not assigned', np.nan)
nbhd_df.Neighborhood = nbhd_df.Neighborhood.combine_first(nbhd_df.Borough)
nbhd_df = nbhd_df.dropna(axis=0, how='any')
#Eliminate future key errors
nbhd_df.head(12)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [4]:
nbhd_df.shape

(211, 3)